In [ ]:
# Check NVIDIA System Management Interface
!nvidia-smi

# If you'd like to see your GPU (maybe to see if it's CUDA compatible)
!lspci | grep -i nvidia

# If you's like to see the location of CUDA
!whereis cuda

# If you'd like to see CUDA version
!nvcc --version

# Check presence of CuDNN 
!whereis cudnn

# Check version of cudnn
!cat /usr/include/cudnn_version.h

In [ ]:
# Check is TensorFlow can access GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()



# Alternatively, check device name
import tensorflow as tf
tf.test.gpu_device_name()

# Check for CUDA
tf.test.is_built_with_cuda()

Clearly, TensorFlow is able access GPU and everything seems normal. What we need to ensure is that the **TensorFlow 2 Object Detection API** is able to access the **GPU**.

**What options do we have?**
1. We could upgrade to CUDA v11.2. I found this <a href="https://medium.com/analytics-vidhya/installing-any-version-of-cuda-on-ubuntu-and-using-tensorflow-and-torch-on-gpu-b1a954500786">blog</a> useful for doing so. But it involves installing 2-3GB CUDA Toolkit from <a href="https://developer.nvidia.com/cuda-11.2.0-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=runfilelocal">here</a> and other complicated software engineering that's especially difficult on Kaggle.
2. We can work with an older version of TensorFlow.

**How do we work with an older version?**
1. Install the latest TF2 OD API (v2.5). You'll also find a directory structure with everything you need.
2. Uninstall tf-models-official v2.5
3. Install tensorflow v2.4
4. Install tf-models-official v2.4 (it is compatible with the hardware/software configuration)

### TensorFlow Object Detection API Installation
#### 1. Downloading the TensorFlow Model Garden
>workspace/

>└─ models/

>        ├─ community/

>        ├─ official/

>        ├─ orbit/

>        ├─ research/

>        └── ...
#### 2. Protobuf Installation/Compilation
#### 3. COCO API installation
#### 4. Install the Object Detection API

In [ ]:
# Download models
!git clone --depth 1 https://github.com/tensorflow/models

# Compile proto files 
%cd models/research
!pip install protobuf
!protoc object_detection/protos/*.proto --python_out=.
%cd ..
%cd ..

# Install cocoapi
!pip install cython 
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ..
%cd ..
!cp -r cocoapi/PythonAPI/pycocotools models/research/

# Install object detection api
%cd models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ..
%cd ..

In [ ]:
# View tensorflow installation
!pip freeze | grep tensorflow

# Uninstall version 2.5
!pip uninstall tf-models-official --yes
# Install version 2.4
!pip install tensorflow==2.4
!pip install tf-models-official==2.4

In [ ]:
# View tensorflow installation
!pip freeze | grep tensorflow

In [ ]:
# Verify TensorFlow 2 Object Detection API installation (optional)
%cd models/research
!python object_detection/builders/model_builder_tf2_test.py
%cd ..
%cd ..

In [ ]:
# Making Directories 
!mkdir workspace
%cd workspace
!mkdir annotations
!mkdir script
!mkdir csv_files
!mkdir pre_trained_models
!mkdir models

In [ ]:
# Skip this..........if you already having tf.records generated and uploaded to kaggle/input/**tf_records folder

# or else Run this following code
# make sure you have the necessary kaggle/input/folders are uploaded properly

# copy the scripts file uploaded to kaggle/input/scripts to kaggle/.../workspace/scripts/
%cd script
!cp /kaggle/input/covid-scripts/generate_tfrecord_csv_to_tfrecord.py .
!ls
%cd ..

# Copy the csv files uploaded to kaggle/input/**csv_files to kaggle/working/workspace/csv_files folders
%cd csv_files
!cp /kaggle/input/covid-csv/X_train.csv  .
!cp /kaggle/input/covid-csv/X_test.csv  .
!cp /kaggle/input/covid-csv/train_data_csv.csv  .
!ls
%cd ..

# run this code to generate tfrecords --- if tf records are not present in kaggle input or kaggle/working/annotations folders
%cd script
! python generate_tfrecord_csv_to_tfrecord.py --csv_input=/kaggle/working/workspace/csv_files/X_train.csv  --output_path=/kaggle/working/workspace/annotations/train.record  --image_dir=""
! python generate_tfrecord_csv_to_tfrecord.py --csv_input=/kaggle/working/workspace/csv_files/X_test.csv  --output_path=/kaggle/working/workspace/annotations/test.record  --image_dir=""

%cd ..
%cd annotations
!cp /kaggle/input/covid-scripts/covid_label_map.pbtxt  .
!ls
%cd ..

In [ ]:
# Run this only if tf.record file are already uploaded to kaggle/input/**tfrecords/folder
# copy the available file to appropriate kaggle/working/***   folders

# skip this if you are generating tf.records here on the run.....
%cd annotations
!cp /kaggle/input/covid-scripts/covid_label_map.pbtxt . 
!cp /kaggle/input/covid-scripts/covid_label_map.pbtxt . 
!cp /kaggle/input/covid-scripts/covid_label_map.pbtxt . 

!ls
%cd ..

In [ ]:
%cd pre_trained_models
# Choose model of your choice and dpaste the download link kere 
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
%cd ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
!ls
%cd ..
%cd ..

In [ ]:
# copying uploaded config file from kaggle/input/**config folder
%cd models
!mkdir ssd_resnet50_v1_fpn
%cd ssd_resnet50_v1_fpn
!cp /kaggle/input/resnet-sheri-config/pipeline.config  .
!ls
%cd ..
%cd ..

In [ ]:
# Make sure that your are in kaggle/working/workspace
# execute the following commands

In [ ]:
!cp  /kaggle/working/models/research/object_detection/model_main_tf2.py  .

In [ ]:
!python model_main_tf2.py --model_dir=models/ssd_resnet50_v1_fpn --pipeline_config_path=models/ssd_resnet50_v1_fpn/pipeline.config